# F1 Data Analysis

idee:
- pulire i dati dei giri trovando un modo di valutare gli errori (tempi che oscillano possno essere dovuti al graining)


In [ ]:
import fastf1
import pandas as pd

FP1 = fastf1.get_session(2025, 'Australia','FP1')
FP2 = fastf1.get_session(2025, 'Australia','FP2')
FP3 = fastf1.get_session(2025, 'Australia','FP3')
FP1.load()
FP2.load()
FP3.load()
free_practice = {'FP1': FP1, 'FP2': FP2, 'FP3': FP3}

In [ ]:
FP1.laps.pick_drivers('NOR').keys()

In [ ]:
def get_free_practice(driver):
    dataset = pd.DataFrame()
    for i in range(1,4):
        tmp = free_practice[f'FP{i}'].laps.pick_drivers(driver)
        tmp['Session'] = f'FP{i}'
        dataset = pd.concat([dataset, tmp])
    return dataset

laps = get_free_practice('LEC')

# we change Stint to track different free practice and stint togheter
max_stints = laps.groupby('Session')['Stint'].max().values
laps['Stint'] = laps['Stint'] + laps['Session'].apply(lambda x: 0 if x == 'FP1' else max_stints[0] if x == 'FP2' else max_stints[0] + max_stints[1])

# we introduce a variable to track the number of laps in a stint
laps['LapsInStint'] = laps.groupby('Stint')['LapNumber'].transform('count')

#we change trackstatus to trackclear wich identifies if the track is clear or not
laps['TrackStatus'] = laps['TrackStatus'].apply(lambda x: True if x == '1' else False)
laps.rename(columns={'TrackStatus': 'TrackClear'}, inplace=True)

# stops are handled by 'IsAccurate'
valid_laps = laps[laps['IsAccurate'] == True]

valid_laps=valid_laps[['LapTime', 'LapNumber', 'Stint','LapsInStint', 'Sector1Time', 'Sector2Time', 'Sector3Time', 'Compound', 'TyreLife', 'FreshTyre', 'TrackClear', 'Deleted', 'Session']]
laps=laps[['LapTime', 'LapNumber', 'Stint','LapsInStint', 'Sector1Time', 'Sector2Time', 'Sector3Time', 'Compound', 'TyreLife', 'FreshTyre', 'TrackClear', 'Deleted', 'Session', 'IsAccurate']]